### Import Dependencies

In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import cv2
import os
from os import listdir
import re
import gc
import pydicom
import datetime
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import dask.array as da
import warnings

### Load Bounding_box File

In [20]:
train_bbox = pd.read_csv('G:/Data/Spine_Data/train_bounding_boxes.csv')
train_bbox

,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
2,1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
3,1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
4,1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137
...,...,...,...,...,...,...
7212,1.2.826.0.1.3680043.9940,297.23186,115.53983,85.18228,66.52623,140
7213,1.2.826.0.1.3680043.9940,298.00000,117.00000,86.00000,61.00000,141
7214,1.2.826.0.1.3680043.9940,298.00000,119.00000,87.00000,58.00000,142
7215,1.2.826.0.1.3680043.9940,299.00000,120.00000,89.00000,56.00000,143


In [46]:
pd=train_bbox[train_bbox['StudyInstanceUID'] == '1.2.826.0.1.3680043.30985']
pd

,StudyInstanceUID,x,y,width,height,slice_number
5107,1.2.826.0.1.3680043.30985,122.88649,197.67089,70.11171,59.86784,106
5108,1.2.826.0.1.3680043.30985,121.04981,179.00383,75.17721,84.35249,107
5109,1.2.826.0.1.3680043.30985,118.00000,180.00000,78.00000,85.00000,108
5110,1.2.826.0.1.3680043.30985,110.89369,174.09962,84.87207,84.36166,109
5111,1.2.826.0.1.3680043.30985,111.00000,183.00000,83.84326,86.00000,110
5112,1.2.826.0.1.3680043.30985,107.31801,184.65947,81.40996,86.54360,111
5113,1.2.826.0.1.3680043.30985,118.10728,183.90805,66.58822,75.01447,112
5114,1.2.826.0.1.3680043.30985,133.03423,139.34177,102.08813,87.75372,113
5115,1.2.826.0.1.3680043.30985,137.18560,139.91711,91.79099,72.87929,114
5116,1.2.826.0.1.3680043.30985,140.41441,142.22342,93.17478,67.32932,115


### Load Train_image File

In [3]:
train_paths = glob(f'G:/Data/Spine_Data/train_images/*')
training_patient=[]
for path in train_paths:
    training_patient.append((path.rsplit("\\",1)[-1])[:])

In [17]:
# import os
# import shutil
# import pydicom
# import pandas as pd
# from PIL import Image
# import numpy as np

# # Path to the train_images folder
# train_images_folder = 'G:/Data/Spine_Data/train_images/'

# # Path to the train_bounding_boxes.csv file
# train_bbox_file = 'G:/Data/Spine_Data/train_bounding_boxes.csv'

# # Output directory for saving PNG images
# output_dir = 'G:/Data/Spine_Data/output_images/'

# # Read train_bounding_boxes.csv into a DataFrame
# train_bbox = pd.read_csv(train_bbox_file)

# # Extract the StudyInstanceUID and slice_number columns
# study_slices = train_bbox[['StudyInstanceUID', 'slice_number']]

# # Iterate over each StudyInstanceUID in training_patient
# for study_id in training_patient:
#     # Check if the StudyInstanceUID exists in the DataFrame
#     if study_id in study_slices['StudyInstanceUID'].values:
#         # Get the corresponding slices for the current StudyInstanceUID
#         slices = study_slices.loc[study_slices['StudyInstanceUID'] == study_id, 'slice_number']
        
#         # Get the path to the study's image folder
#         study_image_folder = os.path.join(train_images_folder, study_id)
        
#         # Check if the image folder exists
#         if os.path.exists(study_image_folder):
#             # Iterate over the DICOM files in the study's image folder
#             for filename in os.listdir(study_image_folder):
#                 dicom_path = os.path.join(study_image_folder, filename)
                
#                 # Read the DICOM file using pydicom
#                 dicom_data = pydicom.dcmread(dicom_path)
                
#                 # Extract the slice number from the DICOM metadata
#                 slice_number = int(dicom_data.InstanceNumber)
                
#                 # Check if the slice number is in the list of slices
#                 if slice_number in slices:
#                     # Determine the category as 'Fracture' since the slice is in the list
#                     category = 'Fracture'
#                 else:
#                     # Determine the category as 'Without Fracture' since the slice is not in the list
#                     category = 'Without Fracture'
                
#                 # Apply windowing to the DICOM image
#                 window_center = 400
#                 window_width = 1800
#                 image = dicom_data.pixel_array.astype(np.float32)
#                 image_min = window_center - window_width // 2
#                 image_max = window_center + window_width // 2
#                 image = np.clip(image, image_min, image_max)
#                 image = (image - image_min) / (image_max - image_min)
#                 image = (image * 255).astype(np.uint8)
                
#                 # Convert image array to PIL Image
#                 pil_image = Image.fromarray(image)
                
#                 # Save the PIL image as PNG with the StudyInstanceUID as the filename
#                 png_filename = f"{study_id}_{slice_number}.png"
#                 png_path = os.path.join(output_dir, category, png_filename)
#                 pil_image.save(png_path)
                
#                 # Optionally, remove the original DICOM file
#                 # os.remove(dicom_path)


#### Extract Slice_Image from Train_images with Bounding_box as Fracture image and without bounding box as without fracture imgae.

In [18]:
import os
import shutil
import pydicom
import pandas as pd
from PIL import Image
import numpy as np

# Path to the train_images folder
train_images_folder = 'G:/Data/Spine_Data/train_images/'

# Path to the train_bounding_boxes.csv file
train_bbox_file = 'G:/Data/Spine_Data/train_bounding_boxes.csv'

# Output directory for saving PNG images
output_dir = 'G:/Data/Spine_Data/output_images/'

# Read train_bounding_boxes.csv into a DataFrame
train_bbox = pd.read_csv(train_bbox_file)

# Extract the StudyInstanceUID and slice_number columns
study_slices = train_bbox[['StudyInstanceUID', 'slice_number']]

# Get the unique StudyInstanceUID values
unique_study_ids = train_bbox['StudyInstanceUID'].unique()

# Iterate over each StudyInstanceUID
for study_id in unique_study_ids:
    # Get the corresponding slices for the current StudyInstanceUID
    slices = study_slices.loc[study_slices['StudyInstanceUID'] == study_id, 'slice_number']
    
    # Get the path to the study's image folder
    study_image_folder = os.path.join(train_images_folder, study_id)
    
    # Check if the image folder exists
    if os.path.exists(study_image_folder):
        # Iterate over the DICOM files in the study's image folder
        for filename in os.listdir(study_image_folder):
            dicom_path = os.path.join(study_image_folder, filename)
            
            # Read the DICOM file using pydicom
            dicom_data = pydicom.dcmread(dicom_path)
            
            # Extract the slice number from the DICOM metadata
            slice_number = int(dicom_data.InstanceNumber)
            
            # Check if the slice number is in the list of slices
            if slice_number in slices.values:
                # Determine the category as 'Fracture' since the slice is in the list
                category = 'Fracture'
            else:
                # Determine the category as 'Without Fracture' since the slice is not in the list
                category = 'Without Fracture'
            
            # Apply windowing to the DICOM image
            window_center = 500
            window_width = 2000
            image = dicom_data.pixel_array.astype(np.float32)
            image_min = window_center - window_width // 2
            image_max = window_center + window_width // 2
            image = np.clip(image, image_min, image_max)
            image = (image - image_min) / (image_max - image_min)
            image = (image * 255).astype(np.uint8)
            
            # Convert image array to PIL Image
            pil_image = Image.fromarray(image)
            
            # Save the PIL image as PNG with the StudyInstanceUID and slice number as the filename
            png_filename = f"{study_id}_{slice_number}.png"
            png_path = os.path.join(output_dir, category, png_filename)
            pil_image.save(png_path)
            
            # Optionally, remove the original DICOM file
            # os.remove(dicom_path)


### Under sampling

In [49]:
import os
import random
import shutil

# Paths to the fracture and without_fracture folders
# fracture_folder = 'G:/Data/Spine_Data/output_images/Fracture/'
# without_fracture_folder = 'G:/Data/Spine_Data/output_images/Without Fracture/'

# Output folder for undersampled images
undersampled_folder = 'G:/Data/Spine_Data/output_images/undersampled/'

# Number of samples to undersample
num_samples = min(len(os.listdir(fracture_folder)), len(os.listdir(without_fracture_folder)))

# Randomly select a subset of without_fracture samples
selected_without_fracture = random.sample(os.listdir(without_fracture_folder), num_samples)

# Move the selected without_fracture samples to the undersampled folder
for file_name in selected_without_fracture:
    src_path = os.path.join(without_fracture_folder, file_name)
    dst_path = os.path.join(undersampled_folder, file_name)
    shutil.move(src_path, dst_path)
